<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/08_sentiment_analysis_with_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMを使ってみる
* 今日は、とりあえず、LLMを使ってみる。

## 今日のコーディングの目的
* いまどのくらい手軽にLLMを使えるようになっているかを、とりあえず体感する。
* 技術的な詳細は次回以降学んでいくことにして、とにかく使ってみる。

**ランタイムのタイプをGPUに設定しておくこと。**

## 例題: LLMによる感情分析
* 今日は、WRIMEというデータセットを使って、LLMに感情分析させてみる。
* 感情分析とは、テキストが表す感情を分析するタスク。
* 今回は、ポジティブな感情か、ネガティブな感情かの2値分類タスクとして解く。
* LLMとしてはELYZA-japanese-Llama-2-7b-instructを使う。
  * プロンプトを使ったテキスト生成によって感情分析の問題を解く。

## インストール
* Hugging Faceの各種ライブラリを使えば、簡単なコードを書くだけでLLMを使える。

### Transformersライブラリのインストール
* https://huggingface.co/docs/transformers/index

In [ ]:
!pip install transformers

### Datasetsライブラリのインストール
* https://huggingface.co/docs/datasets/index

In [ ]:
!pip install datasets

### Accelerateライブラリのインストール
* https://huggingface.co/docs/accelerate/index

In [ ]:
!pip install accelerate

### AutoGPTQのインストール
* モデルを量子化するときに使うライブラリ。
* 量子化されたモデルを使うときにも必要。
  * https://huggingface.co/docs/optimum/llm_quantization/usage_guides/quantization
  * https://huggingface.co/blog/gptq-integration

In [ ]:
!pip install auto-gptq

**ここでランタイムを再起動する。**

## 準備

In [ ]:
import os
import numpy as np
import torch
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

np.random.seed(0)
torch.manual_seed(0)

## データセット
* Ver. 2 の方を使う。
 * WRIME: 主観と客観の感情分析データセット https://github.com/ids-cv/wrime
* Hugging Faceのdatasets hubに登録されているので、簡単に扱うことができる。

### WRIMEデータセットの取得
* training 30,000件、validation 2,500件、test 2,500件。

In [ ]:
dataset = load_dataset("shunk031/wrime", "ver2")

In [ ]:
dataset

### 2値分類問題への単純化
* word vectorの授業回と同じく、2値分類問題に単純化する。
  * https://github.com/tomonari-masada/course2023-nlp/blob/main/04_word_vectors.ipynb

In [ ]:
tags = ["train", "validation", "test"]

texts = {}
labels = {}
for tag in tags:
  texts[tag] = np.array(dataset[tag]["sentence"])
  labels[tag] = [item["sentiment"] for item in dataset[tag]["avg_readers"]]
  labels[tag] = np.array(labels[tag])

In [ ]:
texts["train"][0], labels["train"][0]

* ラベルがneutralなテキストは使わないことにする。
* そして、以下のように2値分類問題へと単純化する。
  * -2, -1のnegativeなラベルは、0とラベルを付け直す。
  * 1, 2のpositiveなラベルは、1とラベルを付け直す。

In [ ]:
texts_binary = {}
labels_binary = {}
for tag in tags:
  indices = labels[tag] != 0
  texts_binary[tag] = texts[tag][indices]
  labels_binary[tag] = labels[tag][indices]
  labels_binary[tag] = (labels_binary[tag] > 0) * 1

In [ ]:
texts_binary["train"][0], labels_binary["train"][0]

* ラベルから感情を表すテキストへのマッピングをおこなうリストを作っておく。
  * なお、両極の感情をどの単語で表せば良いかについて、特に正解はない。

In [ ]:
label_to_text = ["悲しい", "嬉しい"]

## LLM


* 今回は、ELYZA-japanese-Llama-2-7b-instructを使う。
  * https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-instruct
* だがGoogle Colab無料版では、この元のモデルは大きすぎて使えない・・・。
  * 実は、`from_pretrained`メソッドで・・・
  * `torch_dtype=torch.float16`および`device_map="auto"`と設定すれば・・・
  * GPUのメモリを13GBぐらい使ってギリギリでセーフなのだが・・・・
  * モデルのロードに時間がかかる。
* そこで、量子化された下記のモデルを代わりに使う。
  * https://huggingface.co/TFMC/ELYZA-japanese-Llama-2-7b-instruct-GPTQ-4bit-64g

### ELYZA-japanese-Llama-2-7b-instruct-GPTQ-4bit-64gの取得
* モデルのダウンロードに少し時間がかかる。
* `AutoGPTQForCausalLM`クラスについては、以下を参照。
 * https://github.com/PanQiWei/AutoGPTQ/blob/main/auto_gptq/modeling/auto.py

* safetensorsについては、以下を参照。
 * https://huggingface.co/docs/diffusers/using-diffusers/using_safetensors

* QKV fusionについては、以下を参照。
  * https://www.deepspeed.ai/2020/05/27/fastest-bert-training.html
  * ちなみに、DeepSpeedというライブラリ自体については、[これ](https://www.deepspeed.ai/assets/files/DeepSpeed_Overview_Japanese_2023Jun7th.pdf)を参照。

In [ ]:
from auto_gptq import AutoGPTQForCausalLM

model_name = "TFMC/ELYZA-japanese-Llama-2-7b-instruct-GPTQ-4bit-64g"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoGPTQForCausalLM.from_quantized(
    model_name,
    use_safetensors=True,
    inject_fused_attention=False, # これをつけないとエラーがでる
    device="cuda:0",
    )
model.eval()

In [ ]:
model.model

### In-context learning

* 対義語をELYZAに答えさせてみる。

In [ ]:
text = "Q:高い\nA:低い\n\nQ:大きい\nA:小さい\n\nQ:狭い\nA:広い\n\nQ:少ない\nA:多い\n\nQ:速い\nA:遅い\n\nQ:嬉しい\nA:"
print(text)

In [ ]:
token_ids = tokenizer.encode(text, return_tensors="pt")
print(token_ids)

In [ ]:
output_ids = model.generate(
    input_ids=token_ids.to(model.device),
    max_new_tokens=10,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)
print(output_ids)

In [ ]:
output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True)
print(output)

## 感情分析のプロンプト
* LLMがうまく感情分析をしてくれそうなプロンプトを考える。

* Llama 2系言語モデルのプロンプトのテンプレートは、以下の通り。

In [ ]:
"""
<s>[INST] <<SYS>>

{{ System Prompt }}
<</SYS>>
{{ User Prompt }}
 [/INST]
{{ Model Answer }}
"""

### プロンプト作成用のヘルパ関数

* 上のテンプレートをふまえて、プロンプトを適当に考える。

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]答え："
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"

def make_prompt(text):
  prompt = "「" + text + "」と言っている人の気持ちは、「嬉しい」と「悲しい」のうち、どちらですか。"
  return "{b_inst} {system}{prompt} {e_inst} ".format(
      b_inst=B_INST,
      system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
      prompt=prompt,
      e_inst=E_INST,
      ).strip()

In [ ]:
print(make_prompt(texts_binary["train"][0]))

* トークンID列に変換すると、以下のようになる。

In [ ]:
prompt = make_prompt(texts_binary["train"][0])
tokenizer.encode(prompt, return_tensors="pt")

* 冒頭の1は、`<s>`トークン。

In [ ]:
tokenizer.convert_ids_to_tokens(1)

## 感情分析

* 試みに、訓練データのひとつについて、予測させてみる。

In [ ]:
prompt = make_prompt(texts_binary["train"][0])

with torch.no_grad():
  token_ids = tokenizer.encode(prompt, return_tensors="pt")
  output_ids = model.generate(
      input_ids=token_ids.to(model.device),
      max_new_tokens=10,
      pad_token_id=tokenizer.pad_token_id,
      eos_token_id=tokenizer.eos_token_id,
  )
output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True)
print(f"{prompt}\nprediction:{output}")

In [ ]:
print(f"ground truth:「{label_to_text[labels_binary['train'][0]]}」")

### 感情分析の実行

In [ ]:
for i in range(10):
  print(f'[{i+1}]' + '-'*80)
  prompt = make_prompt(texts_binary["train"][i])
  with torch.no_grad():
    token_ids = tokenizer.encode(prompt, return_tensors="pt")
    output_ids = model.generate(
        input_ids=token_ids.to(model.device),
        max_new_tokens=10,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
  output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True)
  print(f"{prompt}\nprediction:{output}")
  print(f"ground truth:{label_to_text[labels_binary['train'][i]]}")
  print('-'*80)

# 本日の課題
* もっとうまくLLMに感情分析をさせるプロンプトを考えてみよう。